In [1]:
import numpy as np

In [2]:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) >= 0)

# set dimension
n = 10
d = 2

# set lambda
lamb = 10

start = True
while start:
    # generate random X
    X = 0.5 * np.random.rand(n, d)
    X[:, 1] = 3.14 * X[:, 1] + np.sqrt(2)
    
    # compute Q
    Q = 0.5 * np.linalg.inv(X.T.dot(X))

    # test if Q is >= 0
    start = ~is_pos_def(Q)
    
# generate random y
y = np.random.binomial(1, 0.5, (n, 1))

# compute p, A and b
p = - np.linalg.inv(X.T.dot(X)).dot(X.T.dot(y))
A = np.concatenate([np.eye(d), -np.eye(d)], axis=0)
b = lamb * np.ones((2 * d, 1))

# possible starting point
v0 = (lamb / 2) * np.ones((d, 1))

In [3]:
def f(Q, p, A, b, x):
    phi = 0
    for i in range(np.shape(A)[0]):
        if float(b[i, 0] - A[i, :].dot(x)) > 0:
            phi -= np.log(float(b[i, 0] - A[i, :].dot(x)))
        else:
            return np.inf
    return float(x.T.dot(Q.dot(x))) + float(p.T.dot(x)) + phi

def grad_f(Q, p, A, b, x):
    phi = np.zeros_like(A[0, :])
    for i in range(np.shape(A)[0]):
        phi += (1 / float(b[i, 0] - A[i, :].dot(x))) * A[i, :].T
    return (Q.T + Q).dot(x) + p + phi.reshape(len(phi), 1)
        
def hess_f(Q, A, b, x):
    phi = np.zeros_like(np.outer(A[0, :], A[0, :]))
    for i in range(np.shape(A)[0]):
        phi +=  (1 / float(A[i, :].dot(x) - b[i, 0]) ** 2) * np.outer(A[i, :], A[i, :])
    return Q + Q.T + phi

In [138]:
# Le probleme c'est que float(grad_f.T.dot(delta)) > 0 alors que delta est une descente donc ca devrait etre
# < 0 !!!!!!

In [23]:
def backtracking_line_search(Q, p, A, b, x, delta, grad_f):
    alpha, beta, t, k = 0.1, 0.8, 1, 0
    while f(Q, p, A, b, x + t * delta) >= f(Q, p, A, b, x) + alpha * t * float(grad_f.T.dot(delta)) + 1e-10 and k < 1e10:
        t *= beta
        k += 1
        #print(float(grad_f.T.dot(delta)))
        #print(f(Q, p, A, b, x + t * delta), f(Q, p, A, b, x) + alpha * t * float(grad_f.T.dot(delta)))
    return t

def centeringstep(Q, p, A, b, t, v0, eps):
    v_seq = [v0]
    decrement = 1e5
    while decrement / 2 > eps:
        grad = grad_f(t * Q, t * p, A, b, v_seq[-1])
        delta = - np.linalg.inv(hess_f(t * Q, A, b, v_seq[-1])).dot(grad) # newton step
        #print(grad.T.dot(- np.linalg.inv(hess_f(t * Q, A, b, v_seq[-1])).dot(grad)))
        decrement = grad.T.dot(- delta) # decrement
        gamma = backtracking_line_search(Q, p, A, b, v_seq[-1], delta, grad) # backtrackling line search
        v_seq.append(v_seq[-1] + gamma * delta)
        #print(decrement / 2 > eps)
    #print(gamma)
    return v_seq
    
def barrmethod(Q, p, A, b, v0, eps):
    m, v_seq = np.shape(A)[0], [v0]
    t, mu = 1, 10
    while m / t >= eps:
        v_seq.append(centeringstep(Q, p, A, b, t, v_seq[-1], eps)[-1]) # minimize t * f_0 + phi and update v_seq (x_star)
        t *= mu
        #print( m / t)
    return v_seq

In [ ]:
v_seq = barrmethod(Q, p, A, b, v0, 1e-6)
v_seq[-1]

In [125]:
from sklearn import linear_model

clf = linear_model.Lasso(alpha=lamb, fit_intercept=False)
clf.fit(X, y)

print(clf.coef_)

[0.         0.00352388]


In [155]:
grad_f(Q, p, A, b, np.array([[0], [0.00352388]]))

array([[-0.30613   ],
       [-0.00516604]])

In [115]:
def phi(A, b, x):
    if (b - A.dot(x) > 0).all():
        return - np.log(b - A.dot(x)).sum() # phi(x) = - sum log(-f_i(x))
    else:
        return np.inf
#         print(x)
#         raise Exception('problem in log') 

def grad_phi(A, b, x):
    res = np.zeros_like(A[0, :])
    for i in range(np.shape(A)[0]):
          res += - (1 / (A[i, :].dot(x) - b[i])) * A[i, :].T
    return np.reshape(res, (np.shape(A)[1], 1))

def hess_phi(A, b, x):
    res = np.zeros((np.shape(A)[1], np.shape(A)[1]))
    for i in range(np.shape(A)[0]):
          res +=  ((1 / (A[i, :].dot(x) - b[i])) ** 2) * np.outer(A[i, :], A[i, :])
    return res

def f(Q, p, A, b, x):
    return x.T.dot(Q.dot(x)) + p.T.dot(x) + phi(A, b, x)
    
def grad(Q, p, A, b, x):
    return (Q.T + Q).dot(x) + p + grad_phi(A, b, x)
    
def hess(Q, p, A, b, x):
    return Q.T + Q + hess_phi(A, b, x)

def backtracking_line_search(Q, p, A, b, x, delta_x):
    alpha, beta, step, k = 0.1, 0.99, 1, 0
    while f(Q, p, A, b, x + step * delta_x) >= f(Q, p, A, b, x) + alpha * step * grad(Q, p, A, b, x).T.dot(delta_x) and k < 1e10:
        step *= beta
        k += 1
    return step # if k < 1e10 else 1
    
# def newton(Q, p, A, b, x):
#     k, max_iter = 0, 10000
#     while np.linalg.norm(grad(Q, p, A, b, x), ord=2) > 1e-6 and k < max_iter:
#         #print(np.linalg.norm(grad(Q, p, A, b, x), ord=2))
#         delta_x = - np.linalg.inv(hess(Q, p, A, b, x)).dot(grad(Q, p, A, b, x))
#         step_size = 1 #backtracking_line_search(Q, p, A, b, x, delta_x)
#         x += step_size * delta_x
#         k += 1
#     return x

# def barrier(Q, p, A, b, t, v0, eps):
#     m = np.shape(A)[0]
#     v = [v0]
#     mu = 20 
#     k, max_iter = 0, 1e10
#     while k < max_iter:
#         v.append(newton(t * Q, t * p, A, b, v[-1])) # minimize t * f_0 + phi and update v
#         if m / t < eps: # stopping criterion 
#             return v
#         t *= mu # increase t
#         k += 1 # increase k
#         print(k)

In [111]:
v = barrier(Q, p, A, b, 1, v0, 1e-5)

NameError: name 'barrier' is not defined

In [ ]:
v[-1]

In [ ]:
from sklearn import linear_model

clf = linear_model.Lasso(alpha=lamb, fit_intercept=False)
clf.fit(X, y)

print(clf.coef_)